# LangChain Expression Language (LCEL)

A Linguagem de Expressão LangChain, ou LCEL, é uma forma declarativa de **compor cadeias de maneira fácil**. A LCEL foi projetada desde o primeiro dia para suportar a colocação de protótipos em produção, sem a necessidade de alterações no código, desde a cadeia mais simples “prompt + LLM” até as cadeias mais complexas (já vimos pessoas executando com sucesso cadeias LCEL com centenas de etapas em produção). Para destacar alguns dos motivos pelos quais você pode querer usar a LCEL:
- **Suporte a streaming de primeira classe**: menor tempo possível para saída do primeiro token produzidio;
- **Suporte assíncrono**: Qualquer cadeia construída com a LCEL pode ser chamada tanto com a API síncrona;
- **Execução paralela otimizada**: Sempre que suas cadeias LCEL tiverem etapas que podem ser executadas em paralelo, automaticamente é feito isso;
- **Retentativas e fallbacks**: É maneira de tornar suas cadeias mais confiáveis em grande escala, na qual ações alternativas podem ser tomadas no caso de um erro em uma cadeia
- **Acessar resultados intermediários**: auxiliando na depuração de uma cadeia;

## Um exemplo simples de LCEL

O exemplo mais simples que podemos mostrar seria na construção de *prompt + model*

In [2]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model='gpt-3.5-turbo-0125')

In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template('Crie uma frase sobre o seguinte: {assunto}')

In [7]:
chain = prompt | model

In [8]:
chain.invoke({'assunto': 'gatinhos'})

AIMessage(content='Os gatinhos são seres adoráveis que conseguem conquistar nossos corações com seu charme e fofura.', response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 19, 'total_tokens': 47}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b9e445f5-5ba9-4aea-9813-eda76880def5-0')

### Adicinando mais elementos a chain

In [9]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [10]:
chain.invoke({'assunto': 'gatinhos'})

'"Gatinhos são fofos, brincalhões e cheios de amor para dar."'

### ATENÇÃO! A ordem importa

In [11]:
chain = model | prompt | output_parser
chain.invoke({'assunto': 'gatinhos'})

ValueError: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.

### Como eu sei a ordem?

Para atingir o mesmo objetivo sem a criação da chain, os passos que eu deveria seguir seriam:
- Formatar o prompt template
- Enviar o prompt formatado para o modelo
- Fazer o parseamento do saída do modelo
Essa mesma ordem deve ser seguida para não termos erros

In [14]:
input = {'assunto': 'gatinhos'}

In [15]:
prompt_formatado = prompt.invoke(input)
prompt_formatado

ChatPromptValue(messages=[HumanMessage(content='Crie uma frase sobre o seguinte: gatinhos')])

In [16]:
resposta = model.invoke(prompt_formatado)
resposta

AIMessage(content='"Os gatinhos são pequenos seres cheios de charme e fofura, capazes de encher nossos corações de alegria e amor."', response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 19, 'total_tokens': 55}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e88e00e3-b1b4-41cd-aa3b-0abf50eb7bbd-0')

In [17]:
output_parser.invoke(resposta)

'"Os gatinhos são pequenos seres cheios de charme e fofura, capazes de encher nossos corações de alegria e amor."'

### Para não errar a ordem

É importante entendermos que cada componente recebe um tipo de entrada e gera um tipo de saída, e estes tipos precisam casar:

| Component       | Tipo de Entrada                                    | Tipo de Saída        |
|-----------------|-----------------------------------------------------|----------------------|
| Prompt          | Dicionário                                         | PromptValue      |
| ChatModel       | String única, lista de mensagens de chat ou PromptValue | Mensagem de Chat     |
| LLM             | String única, lista de mensagens de chat ou PromptValue | String               |
| OutputParser    | A saída de um LLM ou ChatModel                     | Depende do parser    |
| Retriever       | String única                                       | Lista de Documentos  |
| Tool            | String única ou dicionário, dependendo da ferramenta| Depende da ferramenta|

Podemos verificar isso pelos output e input schemas:

In [18]:
print(prompt.input_schema.schema_json(indent=4))

{
    "title": "PromptInput",
    "type": "object",
    "properties": {
        "assunto": {
            "title": "Assunto",
            "type": "string"
        }
    }
}


In [21]:
chain = prompt | model | output_parser
print(chain.input_schema.schema_json(indent=4))

{
    "title": "PromptInput",
    "type": "object",
    "properties": {
        "assunto": {
            "title": "Assunto",
            "type": "string"
        }
    }
}


### Como eu teria feito com chains clássicas?

In [22]:
from langchain.chains.llm import LLMChain
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(model="gpt-3.5-turbo")
prompt = ChatPromptTemplate.from_template("Crie uma frase sobre o assunto: {assunto}")
output_parser = StrOutputParser()

In [23]:
chain = LLMChain(
    llm=model,
    prompt=prompt,
    output_parser=output_parser
)
chain.invoke({'assunto': 'gatinhos'})

c:\Users\avsoares\OneDrive\Documentos\GitHub\courses\Agents LangChain\scripts\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


{'assunto': 'gatinhos',
 'text': 'Os gatinhos são seres adoráveis, que nos encantam com sua fofura e travessuras.'}